In [167]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

## Ideas

Get data about oscar wins
* 1. Extra data to predict how much I will like something with
* 2. Could build a model to predict which films will win each category in a given year

From the data I provide can determine my favorite actors and directors and then find similar movies that contain those actors/directors.

Get rating for each actor and director listed from the average rating of films they have been in; might also be interesting to have quartiles, max, min, sd of ratings for each actor and director. Also number of movies played by the actors and number of movies directed by the directors.  
* Actors popularity can vary wildly over time, and for example an excellent actor may have needed to take roles in many below-average films at the start of their careers to work their way to better films... how talented an actor actually is and how popular they are at that moment in time might be better predictors of film rating.

## Collect all the film links

In [2]:
genre_dict = {
    "1": "action",
    "2": "animation",
    "4": "arts",
    "5": "classics",
    "6": "comedy",
    "8": "documentary",
    "9": "drama",
    "10": "horror",
    "11": "kids",
    "13": "mystery",
    "18": "romance",
    "14": "scifi"
}

In [46]:
for genre in genre_dict:
    print(genre,genre_dict[genre])
    #make browser
    ua=UserAgent()
    dcap = dict(DesiredCapabilities.PHANTOMJS)
    dcap["phantomjs.page.settings.userAgent"] = (ua.random)
    service_args=['--ssl-protocol=any','--ignore-ssl-errors=true']
    driver = webdriver.Chrome("D:/Documents/DataScience/Portfolio/films/films/chromedriver")

    all_movie = 'https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=100&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres='+str(genre)+'&sortBy=tomato'
    driver.get(all_movie)
    webLink = 'https://www.rottentomatoes.com'
    movies = set()
    
    for i in range (0,60):
        try:
            #print(movies)
            showMoreButton = WebDriverWait(driver, 0.01).until(EC.presence_of_element_located((By.XPATH,'//*[@id="show-more-btn"]/button')))
            #click on the showMore Button
            showMoreButton.click()
            print('click: ',i, end="\r")
        except: None
            #print ('missing show More button')

    html=driver.page_source# get the html
    soup = BeautifulSoup(html, "lxml") # parse the html 
    all_movies=soup.findAll('div', {'class':re.compile('movie_info')}) # get all the review divs

    for x in all_movies:
        link=x.find('a',{'href':re.compile('/m/')})
        movLink = webLink + str(link).split('"')[1]
        #print(movLink)
        movies.add(str(movLink))    #creating the distinct list of movies
        
    try:
        fw=open(genre_dict[genre]+'.txt','w') # output file

        for x in movies:
            fw.write(x +"\n")

        fw.close()
    except:
        print("Error while trying to create movie_link.txt file")

1 action
2 animation
4 arts  20 18
5 classics7
6 comedy59 22 27 36
8 documentary2 22
9 drama 593
10 horror97
11 kids 59
13 mystery
18 romance 14 19
14 scifi59


## Explore all the links

In [2]:
#pageLink = action_link[0].replace('\n','')

def run(url):
  
    pageLink = url.replace('\n','')
    for i in range(0,5):
        try:
            response=requests.get(pageLink,headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36', } )        
            html=response.content
            break
        except Exception as e:# browser.open() threw an exception, the attempt to get the response failed
            None
    try:
        soup = BeautifulSoup(html.decode('ascii', 'ignore'),'html.parser') # parse the html
    except:
        print("Failed to get response for ",pageLink)
        return None
    #for i in range(0,5):
    try:
        description = soup.find('div', {'class':re.compile('movie_synopsis')}).text
    except:
        description = 'NA'
    #for i in range(0,5):
    try:
        box = soup.findAll('div', {'class':re.compile('meta-value')})
        rating = box[0].text

    except:
        rating = 'NA' 

    #for i in range(0,5):
    try:
        box4 = soup.findAll('div', {'class':re.compile('meta-value')})
        director_1 = box4[2].findAll(('a', {'href':re.compile('celebrity')}))
        director1 = director_1[0].text

    except:
        director1 =  'NA'    
    #for i in range(0,5):
    try:
        box5 = soup.findAll('div', {'class':re.compile('meta-value')})
        director_2 = box5[2].findAll(('a', {'href':re.compile('celebrity')}))
        director2 = director_2[1].text

    except:
        director2 = 'NA' 
    #for i in range(0,5):
    try:
        box6 = soup.findAll('div', {'class':re.compile('meta-value')})
        director_3 = box6[2].findAll(('a', {'href':re.compile('celebrity')}))
        director3 = director_3[2].text

    except:
        director3 = 'NA' 
    #for i in range(0,5):
    try:
        box7 = soup.findAll('div', {'class':re.compile('meta-value')})
        writer_1 = box7[3].findAll(('a', {'href':re.compile('celebrity')}))
        writer1 = writer_1[0].text

    except:
        writer1 = 'NA'    
    #for i in range(0,5):         
    try:
        box8 = soup.findAll('div', {'class':re.compile('meta-value')})
        writer_2 = box8[3].findAll(('a', {'href':re.compile('celebrity')}))
        writer2 = writer_2[1].text

    except:
        writer2 = 'NA'   
    #for i in range(0,5): 
    try:
        box9 = soup.findAll('div', {'class':re.compile('meta-value')})
        writer_3 = box9[3].findAll(('a', {'href':re.compile('celebrity')}))
        writer3 = writer_3[2].text

    except:
        writer3 =  'NA'    
    #for i in range(0,5):
    try:
        box10 = soup.findAll('div', {'class':re.compile('meta-value')})
        writer_4 = box10[3].findAll(('a', {'href':re.compile('celebrity')}))

        writer4 = writer_4[3].text

    except:
        writer4 = 'NA' 
    #for i in range(0,5):
    try:
        time1 = soup.findAll('time')
        releasedate = time1[0].text

    except:
        releasedate =  'NA'    
    #for i in range(0,5):
    try:
        time2 = soup.findAll('time')

        streamdate = time2[1].text

    except:
        streamdate = 'NA'  
    #for i in range(0,5):
    try:
        time3 = soup.findAll('time')
        runtime = time3[2].text

    except:
        runtime = 'NA'
    #for i in range(0,5):    
    try:
        box11 = soup.findAll('div', {'class':re.compile('meta-value')})
        studio =  box11[-1].text

    except:
        studio = 'NA' 
    #for i in range(0,5):      
    try:
        cast_1 = soup.findAll('div',{'class':re.compile('cast-item')})
        cast1 =  cast_1[0].find('span').text

    except:
        cast1 = 'NA'  
    #for i in range(0,5):
    try:
        cast_2 = soup.findAll('div',{'class':re.compile('cast-item')})
        cast2 =  cast_2[1].find('span').text

    except:
        cast2 = 'NA' 
    #for i in range(0,5):      
    try:
        cast_3 = soup.findAll('div',{'class':re.compile('cast-item')})
        cast3 =  cast_3[2].find('span').text

    except:
        cast3 = 'NA'  
    #for i in range(0,5):  
    try:
        cast_4= soup.findAll('div',{'class':re.compile('cast-item')})
        cast4 =  cast_4[3].find('span').text

    except:
        cast4 = 'NA'
    #for i in range(0,5):      
    try:
        cast_5 = soup.findAll('div',{'class':re.compile('cast-item')})
        cast5 =  cast_5[4].find('span').text

    except:
        cast5 = 'NA'
    #for i in range(0,5):      
    try:
        cast_6 = soup.findAll('div',{'class':re.compile('cast-item')})
        cast6 =  cast_6[5].find('span').text

    except:
        cast6 = 'NA'
    #for i in range(0,5):      
    try:
        title = soup.find('h1',{'class':re.compile('title')}).text
    except:
        title = 'NA'  
        
    try:
        year = soup.find('span',{'class':re.compile('year')}).text
    except:
        year = 'NA'
        
    try:
        critics_consensus = soup.find('h1',{'class':re.compile('mop-ratings-wrap__title mop-ratings-wrap__title--top')}).parent.p.text
    except:
        critics_consensus = 'NA'
        
    try:
        genre = soup.find('div',{'id':re.compile('movieSynopsis')}).parent.findAll('div',{'class':re.compile('meta-value')})[1].text
    except:
        genre = 'NA'

    try:
        tomatometer = soup.find('a',{'id':re.compile('tomato_meter_link')})
        tomatometer = re.findall(r'\d+', tomatometer.text)[0]
        tomatometer_review_count = soup.find('h3',{'class':re.compile('mop-ratings-wrap__title mop-ratings-wrap__title--small')}).parent.small.contents[0]
        tomatometer_review_count = re.findall(r'\d+', tomatometer_review_count)[0]
        #print(tomatometer,tomatometer_review_count)
        audience_meter = re.findall(r'\d+', soup.find('a',{'href':re.compile('#audience_reviews')}).text)[0]
        audience_review_count = re.findall(r'\d+', soup.find('a',{'href':re.compile('#audience_reviews')}).parent.parent.strong.text)[0]
        #print(audience_meter, audience_review_count)
    except:
        tomatometer = 'NA'; tomatometer_review_count = 'NA'
        audience_meter = 'NA'; audience_review_count = 'NA'

    try:
        js_dump = soup.find('script',{'type':re.compile('text/javascript')}).contents[0]
        js_dump_index = js_dump.find("tomatometerAllCritics",120000)
        review_data = js_dump[js_dump_index:js_dump_index+1000]
        avg_score = review_data[review_data.find("avgScore")+10:review_data.find(',"freshCount"')]
        review_data = review_data[review_data.find("tomatometerTopCritics"):-1]
        avg_score_top_critics = review_data[review_data.find("avgScore")+10:review_data.find(',"freshCount"')]
        top_critics_review_count = review_data[review_data.find("topReviews")+12:review_data.find("}")]
        review_data = review_data[review_data.find("audienceAll"):-1]
        avg_score_audience =  review_data[review_data.find("averageRating")+16:review_data.find("scoreSentiment")-3]
        audience_liked_count = review_data[review_data.find("likedCount")+12:review_data.find("notLikedCount")-2]
        audience_disliked_count = review_data[review_data.find("notLikedCount")+15:review_data.find("}")]
        review_data = review_data[review_data.find("wantToSeeData"):-1]
        want_to_see_count = review_data[review_data.find("wantToSeeCount")+16:review_data.find("ratingsStartDate")-2]
        #print(js_dump_index,avg_score,avg_score_top_critics,top_critics_review_count,avg_score_audience,audience_liked_count,audience_disliked_count,want_to_see_count)
    except:
        js_dump_index = 'NA'; avg_score = 'NA'; avg_score_top_critics = 'NA'; top_critics_review_count = 'NA'; avg_score_audience = 'NA'
        audience_liked_count = 'NA'; audience_disliked_count = 'NA'; want_to_see_count = 'NA'

    return (description,rating,director1,director2,director3,writer1,writer2,writer3,writer4,releasedate,streamdate,runtime,
            studio,cast1,cast2,cast3,cast4,cast5,cast6,title,year,critics_consensus,genre,
            tomatometer,tomatometer_review_count,audience_meter,audience_review_count,
            js_dump_index,avg_score,avg_score_top_critics,top_critics_review_count,avg_score_audience,audience_liked_count,audience_disliked_count,want_to_see_count,pageLink)


In [ ]:
col_names = [
    'description','rating','director1','director2','director3','writer1','writer2','writer3','writer4','releasedate','streamdate','runtime',
            'studio','cast1','cast2','cast3','cast4','cast5','cast6','title','year','critics_consensus','genre',
            'tomatometer','tomatometer_review_count','audience_meter','audience_review_count',
            'js_dump_index','avg_score','avg_score_top_critics','top_critics_review_count','avg_score_audience','audience_liked_count','audience_disliked_count','want_to_see_count','pageLink'
]

for genre in genre_dict:
    print(genre,genre_dict[genre])
    print()
    film_links = open("D:/Documents/DataScience/Portfolio/films/films/"+genre_dict[genre]+".txt").readlines()
    data_store_genre = []
    for i in range(0,len(film_links)):
        print(i," / ",len(film_links),end="\r")
        data_store_genre.append(run(film_links[i]))
        
    df_genre = pd.DataFrame(data_store_genre)
    df_genre.columns = col_names
    path = r'D:/Documents/DataScience/Portfolio/films/films/'
    df_genre.to_csv(os.path.join(path, genre_dict[genre]+r'.csv'), index=False)

1 action

2 animation13

4 arts  585



In [3]:
genre_dict = {
#    "1": "action",
#    "2": "animation",
#    "4": "arts",
#    "5": "classics",
#    "6": "comedy",
    "8": "documentary",
    "9": "drama",
    "10": "horror",
    "11": "kids",
    "13": "mystery",
    "18": "romance",
    "14": "scifi"
}

col_names = [
    'description','rating','director1','director2','director3','writer1','writer2','writer3','writer4','releasedate','streamdate','runtime',
            'studio','cast1','cast2','cast3','cast4','cast5','cast6','title','year','critics_consensus','genre',
            'tomatometer','tomatometer_review_count','audience_meter','audience_review_count',
            'js_dump_index','avg_score','avg_score_top_critics','top_critics_review_count','avg_score_audience','audience_liked_count','audience_disliked_count','want_to_see_count','pageLink'
]

for genre in genre_dict:
    print(genre,genre_dict[genre])
    print()
    film_links = open("D:/Documents/DataScience/Portfolio/films/films/"+genre_dict[genre]+".txt").readlines()
    data_store_genre = []
    for i in range(0,len(film_links)):
        print(i," / ",len(film_links),end="\r")
        data_store_genre.append(run(film_links[i]))
        
    df_genre = pd.DataFrame(data_store_genre)
    df_genre.columns = col_names
    path = r'D:/Documents/DataScience/Portfolio/films/films/'
    df_genre.to_csv(os.path.join(path, genre_dict[genre]+r'.csv'), index=False)

8 documentary

9 drama  1617

10 horror1565

11 kids  1498

13 mystery036

18 romance621

14 scifi 1638



In [19]:
#col_names = [
#    'description','rating','director1','director2','director3','writer1','writer2','writer3','writer4','releasedate','streamdate','runtime',
#            'studio','cast1','cast2','cast3','cast4','cast5','cast6','title','year',
#            'tomatometer','tomatometer_review_count','audience_meter','audience_review_count',
#            'js_dump_index','avg_score','avg_score_top_critics','top_critics_review_count','avg_score_audience','audience_liked_count','audience_disliked_count','want_to_see_count'
#]
#film_links = open("D:/Documents/DataScience/Portfolio/films/films/"+genre_dict["1"]+".txt").readlines()
#data_store_genre = []
#for i in range(0,5):
#    print(i," / ",len(film_links),end="\r")
#    data_store_genre.append(run(film_links[i]))
#df_genre = pd.DataFrame(data_store_genre)
#df_genre.columns = col_names
#df_genre

,description,rating,director1,director2,director3,writer1,writer2,writer3,writer4,releasedate,streamdate,runtime,studio,cast1,cast2,cast3,cast4,cast5,cast6,title,year,tomatometer,tomatometer_review_count,audience_meter,audience_review_count,js_dump_index,avg_score,avg_score_top_critics,top_critics_review_count,avg_score_audience,audience_liked_count,audience_disliked_count,want_to_see_count
0,\n After losing their family ho...,R (for violence)\n,Rachid Bouchareb,NA,NA,Rachid Bouchareb,Olivier Lorelle,NA,NA,"Nov 3, 2010\n","Aug 2, 2011\n",\n 137 minutes\n ...,\n Independent Pictures...,\n Sami Bouajila\n,\n Jamel Debbouze\n,\n Roschdy Zem\n,\n Bernard Blancan\n,\n Larbi Zekkal\n,\n Sabrina Seyvecou\n,\n Outside the ...,2010,76,41,65,678,134733,6.46,6.41,20,3.5,168,91,220
1,\n When the scout fairies grow ...,G\n,Steve Loter,NA,NA,Steve Loter,Tom Rogers,Robert Schooley,Mark McCorkle,"Mar 3, 2015\n",\n 76 minutes\n ...,NA,\n DisneyToon Studios\n...,\n Ginnifer Goodwin\n,\n Mae Whitman\n,\n Pamela Adlon\n,\n Lucy Liu\n,\n Megan Hilty\n,\n Raven-Symon\n,NA,NA,75,20,76,1,133438,5.59,5,8,3.98,436,137,708
2,\n The most famed and well-rega...,R\n,Werner Herzog,NA,NA,Werner Herzog,NA,NA,NA,"Dec 29, 1972\n","Oct 24, 2000\n",\n 90 minutes\n ...,\n New Yorker Films\n ...,\n Klaus Kinski\n,\n Ruy Guerra\n,\n Del Negro\n,\n Helena Rojo\n,\n Cecilia Rivera\n,\n Peter Berling\n,"\n Aguirre, the...",1972,98,46,91,24,134870,9.11,9.22,10,4.35,4904,482,2693
3,"\n A talented, young getaway dr...",R (for violence and language throughout.)\n ...,Edgar Wright,NA,NA,Edgar Wright,NA,NA,NA,"Jun 28, 2017\n","Oct 10, 2017\n",\n 113 minutes\n ...,\n Sony Pictures\n ...,\n Ansel Elgort\n,\n Kevin Spacey\n,\n Lily James\n,\n Jon Bernthal\n,\n Eiza Gonzalez\n,\n Jon Hamm\n,NA,NA,93,373,86,62,138110,8.03,8.44,52,4.13,26931,4395,28109
4,\n We all wish for the chance t...,PG-13\n,Jay Oliva,NA,NA,James Krieg,Geoff Johns,Jerry Siegel,NA,"Jul 30, 2013\n",\n 95 minutes\n ...,NA,\n WARNER BROTHERS PICT...,\n Justin Chambers\n,\n Cary Elwes\n,\n Nathan Fillion\n,\n Ron Perlman\n,\n C. Thomas Howell\n,\n Kevin Mckidd\n,\n DCU: Justice...,2013,100,5,93,5,133388,7.62,0,0,4.38,4310,349,893


In [173]:
#for i in range(0,25):
#    c = []
#    for i in range(0,len(action)):
#        if action[i][0] == 'NA':
#            c.append(i) # find rows where description is na
#
#    for item in c:
#        action[item] = run(action_link[item])

## Analysis: preprocessing

In [6]:
genre_dict = {
    "1": "action",
    "2": "animation",
    "4": "arts",
    "5": "classics",
    "6": "comedy",
    "8": "documentary",
    "9": "drama",
    "10": "horror",
    "11": "kids",
    "13": "mystery",
    "18": "romance",
    "14": "scifi"
}

path = r'D:/Documents/DataScience/Portfolio/films/films/genres_data/'
dfs = dict()
for genre in genre_dict:
    dfs[genre] = pd.read_csv(os.path.join(path, genre_dict[genre]+r'.csv'))
df = pd.concat(dfs.values()).reset_index()

In [32]:
def clean_text(df, text_field):
    mask = ~df[text_field].isna()
    #df.loc[mask,text_field] = df.loc[mask,text_field].str.lower()
    #df.loc[mask,text_field] = df.loc[mask,text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    #df.loc[mask,text_field] = df.loc[mask,text_field].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    # remove newlines
    df.loc[mask,text_field] = df.loc[mask,text_field].replace('\n','')
    
    # remove duplicate whitespace (and trailing and leading spaces)
    df.loc[mask,text_field] = df.loc[mask,text_field].apply(lambda elem: re.sub("\s\s+" , " ", elem.rstrip().lstrip()))
    
    return df
cols_string = ['description','rating','director1','director2','director3','writer1','writer2','writer3','writer4','releasedate',
              'streamdate','runtime','studio','cast1','cast2','cast3','cast4','cast5','cast6','title','critics_consensus','genre']

df_clean = df[~df.title.isna()].copy(deep=True)
df_clean = df_clean.drop_duplicates(subset=['title'])
for col in cols_string:
    df_clean = clean_text(df_clean, col)

In [33]:
# Get how often each genre occurs; one-hot encode each genre
genre_breakdown = pd.DataFrame(df_clean['genre'].str.split(',', expand=True).stack().str.lstrip().value_counts()).reset_index()
genre_breakdown.columns = ['genre','frequency']
genre_breakdown['genre_col'] = genre_breakdown['genre'].str.lower().apply(lambda elem: re.sub("\s+" , "", elem))

for idx, (genre,frequency,genre_col) in genre_breakdown.iterrows():
    df_clean[genre_col] = df_clean['genre'].str.contains(genre)

In [34]:
cols_numeric = ['tomatometer','tomatometer_review_count','audience_meter','audience_review_count','avg_score','avg_score_top_critics',
               'top_critics_review_count','avg_score_audience','audience_liked_count','audience_disliked_count','want_to_see_count']
for col in cols_numeric:
    df_clean[col] = pd.to_numeric(df_clean[col],errors='coerce')

In [63]:
scoring_col = 'avg_score_top_critics' #avg_score
index_cols = ['cast1','cast2','cast3','cast4','cast5','cast6']
film_scoring_col_mean = np.nanmean(df_clean[scoring_col])

dfs_actors = list()
for col in index_cols:
    dfs_actors.append(df_clean[[col,scoring_col]].copy(deep=True).rename(columns={col:'cast'}))
df_actors = pd.concat(dfs_actors)

def actor_score(values):
    prior_size = 5; prior_value = film_scoring_col_mean
    return (len(values)*np.nanmean(values) + prior_size*prior_value)/(len(values)+prior_size)

df_actors = df_actors.groupby(['cast']).agg({
    scoring_col: ['mean','count','std','median',actor_score]
}).sort_values((scoring_col,'actor_score'),ascending=False)

In [86]:
scoring_col = 'avg_score_top_critics' #avg_score
index_cols = ['cast1','cast2','cast3','cast4','cast5','cast6']

def create_person_score_subtable(scoring_col, index_cols, prior_size):
    film_scoring_col_mean = np.nanmean(df_clean[scoring_col])

    dfs_temp = list()
    for col in index_cols:
        dfs_temp.append(df_clean[[col,scoring_col]].copy(deep=True).rename(columns={col:'cast'}))
    df_temp = pd.concat(dfs_temp)

    def score(values):
        #prior_size = 5
        prior_value = film_scoring_col_mean
        return (len(values)*np.nanmean(values) + prior_size*prior_value)/(len(values)+prior_size)

    df_temp = df_temp.groupby(['cast']).agg({
        scoring_col: ['mean','count','std','median',score]
    }).sort_values((scoring_col,'score'),ascending=False)
    
    return df_temp

# avg_score_top_critics has issue that when have no top critic reviews gives score of 0
df_actors = create_person_score_subtable('avg_score', ['cast1','cast2','cast3','cast4','cast5','cast6'], 5)
df_directors = create_person_score_subtable('avg_score', ['director1','director2','director3'], 3)
df_writers = create_person_score_subtable('avg_score', ['writer1','writer2','writer3','writer4'], 5)
df_directors

<ipython-input-86-7803e68e5db3>:15: RuntimeWarning: Mean of empty slice
  return (len(values)*np.nanmean(values) + prior_size*prior_value)/(len(values)+prior_size)


avg_score                                 
                           mean count       std median     score
cast                                                            
Buster Keaton          8.710000     7  1.082051  8.970  8.053534
Charles Chaplin        8.522222     9  0.559079  8.570  8.022112
Luis Buuel             8.658571     7  0.380369  8.630  8.017534
Yasujiro Ozu           8.876000     5  0.546059  8.890  7.993168
Akira Kurosawa         8.350000    12  0.873998  8.655  7.984356
...                         ...   ...       ...    ...       ...
Michael Gottlieb       3.113333     3  0.574659  3.430  4.817557
Raja Gosnell           3.843333     6  0.517288  4.005  4.736149
John R. Cherry III     3.532000     5  0.665485  3.500  4.653168
Jay Karas              3.242500     4  3.943166  2.485  4.647906
Shannon Sun-Higginson       NaN     0       NaN    NaN       NaN

[6350 rows x 5 columns]

In [157]:
df_with_scores = df_clean.copy(deep=True)

def multindex_rename(df,new_names):
    df.columns = new_names
    return df

score_col_dict = {
    'director_score': (df_directors, ['director1','director2','director3']),
    'writer_score': (df_writers, ['writer1','writer2','writer3','writer4']),
    'cast_score': (df_actors, ['cast1','cast2','cast3','cast4','cast5','cast6']),
}
for score_col in score_col_dict.keys():
    for score_col_num in score_col_dict[score_col][1]:
        df_score_col = score_col_dict[score_col][0]
        df_with_scores = pd.merge(
            df_with_scores,
            multindex_rename(df_score_col[[('avg_score','score')]],[score_col_num+"_"+score_col]),
            left_on = [score_col_num], right_index = True, how='left'
        )
    df_with_scores[score_col] = df_with_scores[[x + "_" + score_col for x in score_col_dict[score_col][1]]].apply(lambda x: x.mean(), axis=1)

In [131]:
from fuzzywuzzy import fuzz

In [171]:
path = r'D:/Documents/DataScience/Portfolio/films/films/'
df_personal_ratings = pd.read_csv(os.path.join(path, 'films_list_working_film_list'+r'.csv'))
df_personal_ratings['film_title'] = df_personal_ratings['Film']
# Removing the year in brackets after film title, however this means any film titles that actually include brackets won't match
df_personal_ratings['film_title'] = df_personal_ratings['film_title'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).rstrip())
df_personal_ratings['is_in_main_db'] = df_personal_ratings['film_title']\
    .apply(lambda x: any(df_with_scores['title'].apply(lambda x: x.lower()).eq(x.lower())))

df_personal_ratings['best_match_ratio'] = np.nan
df_personal_ratings['best_match_title'] = np.nan
for idx, row in df_personal_ratings.iterrows():
    film_title = row['film_title']
    #print(film_title)
    ratios = df_with_scores['title'].apply(lambda y: fuzz.ratio(film_title.lower(), y.lower()))
    #ratios = df_with_scores['title'].apply(lambda y: fuzz.token_set_ratio(film_title.lower(), y.lower()))
    max_ratio = max(ratios)
    best_title_match = df_with_scores.loc[np.argmax(ratios),'title']
    df_personal_ratings.loc[idx,'best_match_ratio'] = max_ratio
    df_personal_ratings.loc[idx,'best_match_title'] = best_title_match

In [172]:
df_personal_ratings = df_personal_ratings[df_personal_ratings.best_match_ratio >= 85]
df_personal_ratings = df_personal_ratings[~df_personal_ratings.film_title.isin(['City of God'])]

In [174]:
df_with_personal_ratings = pd.merge(
    df_with_scores,
    df_personal_ratings[['best_match_title','Rating']].rename(columns={'Rating':'ben_rating'}),
    left_on = ['title'], right_on = ['best_match_title'], how='left'
)

In [175]:
df_with_personal_ratings.sort_values('ben_rating',ascending=False).head(100)

,level_0,index,description,rating,director1,director2,director3,writer1,writer2,writer3,writer4,releasedate,streamdate,runtime,studio,cast1,cast2,cast3,cast4,cast5,cast6,title,year,critics_consensus,genre,tomatometer,tomatometer_review_count,audience_meter,audience_review_count,js_dump_index,avg_score,avg_score_top_critics,top_critics_review_count,avg_score_audience,audience_liked_count,audience_disliked_count,want_to_see_count,pageLink,drama,comedy,mystery&suspense,action&adventure,arthouse&international,sciencefiction&fantasy,romance,documentary,horror,classics,kids&family,specialinterest,musical&performingarts,animation,television,western,sports&fitness,cultmovies,faith&spirituality,gay&lesbian,anime&manga,director1_director_score,director2_director_score,director3_director_score,director_score,writer1_writer_score,writer2_writer_score,writer3_writer_score,writer4_writer_score,writer_score,cast1_cast_score,cast2_cast_score,cast3_cast_score,cast4_cast_score,cast5_cast_score,cast6_cast_score,cast_score,best_match_title,ben_rating
6316,8127,405,"Based on a true story, Steven Spielberg's Schi...",R,Steven Spielberg,NaN,NaN,Steven Zaillian,NaN,NaN,NaN,"Dec 15, 1993","Mar 9, 2004",196 minutes,Universal Pictures,Liam Neeson,Ben Kingsley,Ralph Fiennes,Caroline Goodall,Jonathan Sagall,Jonathan Sagalle,Schindler's List,1993.0,Schindler's List blends the abject horror of t...,Drama,97.0,97.0,97.0,411.0,138137.0,9.02,9.04,24.0,4.62,80519.0,2571.0,76683.0,https://www.rottentomatoes.com/m/schindlers_list,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,7.398115,NaN,NaN,7.398115,7.083927,NaN,NaN,NaN,7.083927,6.213695,6.374445,7.093716,6.338890,6.938151,6.938151,6.649508,Schindler's List,10.0
6699,9265,1543,A jury argues a case in a stuffy room on a hot...,PG-13,William Friedkin,NaN,NaN,Reginald Rose,NaN,NaN,NaN,"Aug 17, 1997","May 8, 1998",117 minutes,Orion Home Video,George C. Scott,Jack Lemmon,Hume Cronyn,Tony Danza,Courtney B. Vance,Mykelti Williamson,12 Angry Men,1997.0,No consensus yet.,"Drama, Television",92.0,12.0,87.0,20.0,132978.0,7.34,0.00,0.0,4.16,1696.0,253.0,3946.0,https://www.rottentomatoes.com/m/1079908-12_an...,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,6.535940,NaN,NaN,6.535940,7.192363,NaN,NaN,NaN,7.192363,6.945495,7.266138,6.857260,6.676986,6.551890,6.423537,6.786884,12 Angry Men,10.0
4091,4839,195,From its opening multi-language titles (that s...,PG,Terry Gilliam,Terry Jones,NaN,Graham Chapman,John Cleese,Eric Idle,Terry Gilliam,"Jan 1, 1975","Sep 7, 1999",92 minutes,Almi Cinema 5,Graham Chapman,John Cleese,Eric Idle,Terry Jones,Connie Booth,Neil Innes,Monty Python and the Holy Grail,1975.0,A cult classic as gut-bustingly hilarious as i...,Comedy,97.0,76.0,95.0,644.0,141648.0,8.49,8.67,20.0,4.51,74554.0,4049.0,67514.0,https://www.rottentomatoes.com/m/monty_python_...,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6.779023,6.456918,NaN,6.617970,6.796545,7.234446,7.153890,6.861807,7.011672,6.848890,6.196630,6.486182,6.616264,6.849817,6.849817,6.641267,Monty Python and the Holy Grail,10.0
6635,9059,1337,The most brilliant mind at America's top unive...,"R (for strong language, including some sex-rel...",Gus Van Sant,NaN,NaN,Ben Affleck,Matt Damon,Gus Van Sant,NaN,"Dec 5, 1997","Dec 8, 1998",126 minutes,Miramax Films,Matt Damon,Robin Williams,Ben Affleck,Minnie Driver,Stellan Skarsgard,Casey Affleck,Good Will Hunting,1997.0,"It follows a predictable narrative arc, but Go...",Drama,97.0,75.0,94.0,364.0,135343.0,8.05,7.92,20.0,4.41,62047.0,3622.0,44359.0,https://www.rottentomatoes.com/m/good_will_hun...,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,7.300534,NaN,NaN,7.300534,7.014863,6.776484,7.076264,NaN,6.955870,6.9

In [125]:
df_with_scores['title'].apply(lambda x: x.lower())

0                            outside the law (hors-la-loi)
1              tinkerbell and the legend of the neverbeast
2        aguirre, the wrath of god (aguirre, der zorn g...
3                                              baby driver
4              dcu: justice league: the flashpoint paradox
                               ...                        
16751    lawnmower man 2 - jobe's war (lawnmower man 2:...
16753                                   the age of adaline
16755                                    the fifth element
16758                                                   io
16764                                       the terminator
Name: title, Length: 10406, dtype: object

In [165]:
df_with_scores[df_with_scores.title.str.contains('Lon: The Professional')]

,level_0,index,description,rating,director1,director2,director3,writer1,writer2,writer3,writer4,releasedate,streamdate,runtime,studio,cast1,cast2,cast3,cast4,cast5,cast6,title,year,critics_consensus,genre,tomatometer,tomatometer_review_count,audience_meter,audience_review_count,js_dump_index,avg_score,avg_score_top_critics,top_critics_review_count,avg_score_audience,audience_liked_count,audience_disliked_count,want_to_see_count,pageLink,drama,comedy,mystery&suspense,action&adventure,arthouse&international,sciencefiction&fantasy,romance,documentary,horror,classics,kids&family,specialinterest,musical&performingarts,animation,television,western,sports&fitness,cultmovies,faith&spirituality,gay&lesbian,anime&manga,director1_director_score,director2_director_score,director3_director_score,director_score,writer1_writer_score,writer2_writer_score,writer3_writer_score,writer4_writer_score,writer_score,cast1_cast_score,cast2_cast_score,cast3_cast_score,cast4_cast_score,cast5_cast_score,cast6_cast_score,cast_score
8450,12193,372,As visually stylish as it is graphically viole...,"R (for scenes of strong graphic violence, and ...",Luc Besson,NaN,NaN,Luc Besson,NaN,NaN,NaN,"Nov 18, 1994","Aug 15, 2000",104 minutes,Columbia Pictures,Jean Reno,Gary Oldman,Natalie Portman,Danny Aiello,Peter Appel,Michael Badalucco,Lon: The Professional,NaN,Pivoting on the unusual relationship between s...,"Action & Adventure, Drama, Mystery & Suspense",73.0,64.0,95.0,274.0,137878.0,6.85,5.54,18.0,4.41,51685.0,2780.0,27051.0,https://www.rottentomatoes.com/m/leon_the_prof...,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6.107534,NaN,NaN,6.107534,6.033828,NaN,NaN,NaN,6.033828,5.959927,6.589617,6.375605,6.675355,6.589843,6.646986,6.472889


In [14]:
df_clean.cast6.value_counts()

Tony Shalhoub       15
Albert Basserman    15
Michael Gambon      14
Wallace Shawn       13
Emma Thompson       11
                    ..
Adam Godley          1
Tina Benko           1
James Gleason        1
Maureen Keiller      1
Dolly Haas           1
Name: cast6, Length: 7908, dtype: int64

'\n                After losing their family home in Algeria, three brothers are scattered across the globe. Messaoud joins the French army fighting in Indochina; Abdelkader becomes a leader of the Algerian independence movement; Sad moves to Paris to make his fortune in the shady clubs and boxing halls of Pigalle. Gradually, their interconnecting destinies reunite them in the French capital, where freedom is a battle to be fought and won.-- (C) Official Site\n            '

'                after losing their family home in algeria three brothers are scattered across the globe messaoud joins the french army fighting in indochina abdelkader becomes a leader of the algerian independence movement sad moves to paris to make his fortune in the shady clubs and boxing halls of pigalle gradually their interconnecting destinies reunite them in the french capital where freedom is a battle to be fought and won c official site            '